In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [13]:
# Load and clean data
data = pd.read_csv('nbaHomeWinLossModelDataset.csv').drop(['Unnamed: 0'], axis=1)
data = data.dropna()

In [14]:
validation = data[data['SEASON'] == '2024-25']  # Predicting 2024-25 season
modelData = data[data['SEASON'] != '2024-25'].sample(frac=1)  # Training on past seasons

In [15]:
# Split into training (past seasons) and validation (2024-25 season)
validation = data[data['SEASON'] == '2024-25']  # Predicting 2024-25 season
modelData = data[data['SEASON'] != '2024-25'].sample(frac=1)  # Training on past seasons.

# Define features and target variable
X = modelData.drop(['HOME_W', 'SEASON'], axis=1)
y = modelData['HOME_W']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Standard Scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# Build Neural Network Model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # First hidden layer
    layers.Dense(32, activation='relu'),  # Second hidden layer
    layers.Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

/Users/adityabhosale/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the Model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\nNeural Network Test Accuracy: {test_acc:.2f}")

Epoch 1/50
3676/3676 ━━━━━━━━━━━━━━━━━━━━ 3s 494us/step - accuracy: 0.8750 - loss: 0.3056 - val_accuracy: 0.9223 - val_loss: 0.2234
Epoch 2/50
3676/3676 ━━━━━━━━━━━━━━━━━━━━ 2s 480us/step - accuracy: 0.9300 - loss: 0.2079 - val_accuracy: 0.9454 - val_loss: 0.1695
Epoch 3/50
3676/3676 ━━━━━━━━━━━━━━━━━━━━ 2s 478us/step - accuracy: 0.9510 - loss: 0.1620 - val_accuracy: 0.9582 - val_loss: 0.1427
Epoch 4/50
3676/3676 ━━━━━━━━━━━━━━━━━━━━ 2s 485us/step - accuracy: 0.9609 - loss: 0.1333 - val_accuracy: 0.9607 - val_loss: 0.1305
Epoch 5/50
3676/3676 ━━━━━━━━━━━━━━━━━━━━ 2s 484us/step - accuracy: 0.9633 - loss: 0.1200 - val_accuracy: 0.9641 - val_loss: 0.1194
Epoch 6/50
3676/3676 ━━━━━━━━━━━━━━━━━━━━ 2s 487us/step - accuracy: 0.9656 - loss: 0.1106 - val_accuracy: 0.9655 - val_loss: 0.1133
Epoch 7/50
3676/3676 ━━━━━━━━━━━━━━━━━━━━ 2s 499us/step - accuracy: 0.9685 - loss: 0.1017 - val_accuracy: 0.9685 - val_loss: 0.1042
Epoch 8/50
3676/3676 ━━━━━━━━━━━━━━━━━━━━ 2s 485us/step - accuracy: 0.9703 -

In [18]:
# Make Predictions on Test Set
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


1811/1811 ━━━━━━━━━━━━━━━━━━━━ 0s 201us/step
              precision    recall  f1-score   support

           0       0.99      0.98      0.98     26481
           1       0.98      0.99      0.99     31446

    accuracy                           0.99     57927
   macro avg       0.99      0.98      0.99     57927
weighted avg       0.99      0.99      0.99     57927



In [20]:
# Transform Validation Set & Predict
scaled_val_data = scaler.transform(validation.drop(['HOME_W', 'SEASON'], axis=1))
y_val_pred = (model.predict(scaled_val_data) > 0.5).astype("int32")
print(classification_report(validation['HOME_W'], y_val_pred))

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step
              precision    recall  f1-score   support

           0       0.53      0.50      0.51       422
           1       0.61      0.64      0.63       526

    accuracy                           0.58       948
   macro avg       0.57      0.57      0.57       948
weighted avg       0.57      0.58      0.58       948



In [21]:
# Predict outcomes for the 2024-25 season
y_pred = model.predict(scaled_val_data)

# Print classification report for the validation set
print(classification_report(validation['HOME_W'], y_pred))

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step


ValueError: Classification metrics can't handle a mix of binary and continuous targets